# Taller de búsqueda de hiperparámetros

En este taller vamos a explorar la búsqueda de hiperparámetros de manera automática.

In [4]:
import time
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [6]:
X, y = load_breast_cancer(return_X_y=True)

In [7]:
test_size=0.25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

Experimento 1

**Ejemplo 1:** encuentra los mejores hiperparámetros usando `GridSearchCV`

In [8]:
clf = DecisionTreeClassifier()

# define los valores que usarás en la búsqueda del
# hiperparametro C
param_grid = {
    "max_depth": (3, 6, 12, 18),
    "min_samples_leaf": (1, 2, 3),
    "criterion": ["gini", "entropy"]
}

# Utiliza GridSearchCV
gs = GridSearchCV(clf, param_grid, scoring="f1", cv=5, verbose=1, n_jobs=-1)
t0 = time.time()
gs.fit(X_train, y_train)
print("Tiempo de búsqueda: {:.3f}s".format(time.time() - t0))

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Tiempo de búsqueda: 8.995s


podemos ver que el atributo `cv_results_` nos entrega los resultados de toda la búsqueda.

In [9]:
dir(gs)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__sklearn_tags__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_refit_for_multimetric',
 '_check_scorers_accept_sample_weight',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_params_html',
 '_get_routed_params_for_fit',
 '_get_scorers',
 '_html_repr',
 '_parameter_constraints',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_run_sea

In [10]:
gs.cv_results_

{'mean_fit_time': array([0.0179215 , 0.02213984, 0.01133819, 0.01356902, 0.01767073,
        0.01046572, 0.00875077, 0.00864305, 0.00851927, 0.00842528,
        0.01013732, 0.0088376 , 0.00793023, 0.00862041, 0.00943451,
        0.03090749, 0.01214275, 0.01093826, 0.01204128, 0.01003661,
        0.01224403, 0.01194715, 0.01013103, 0.01254525]),
 'std_fit_time': array([0.01577559, 0.0191905 , 0.00194217, 0.00739448, 0.01297959,
        0.00508957, 0.00059193, 0.00131871, 0.00158413, 0.00111792,
        0.00330021, 0.00156934, 0.00037758, 0.00057859, 0.00234278,
        0.02194823, 0.00248512, 0.00216237, 0.00508129, 0.00055115,
        0.00112275, 0.00449144, 0.00106893, 0.0029356 ]),
 'mean_score_time': array([0.01186266, 0.00582309, 0.00883784, 0.00944161, 0.00461845,
        0.00372109, 0.00371675, 0.00300484, 0.00321207, 0.00401058,
        0.0049181 , 0.00362206, 0.00361786, 0.00562229, 0.00370727,
        0.00411482, 0.00320892, 0.00350919, 0.00351453, 0.00360918,
        0.003906

Lo más importante es extraer los hiperparámetros del modelo que mejor error en de validación sacaron

In [11]:
gs.best_params_

{'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 1}

También es posible el mejor resultado en la métrica usada

In [12]:
gs.best_score_

np.float64(0.9545438858529135)

Finalmente, es posible extraer directamente un estimador que que ha sido creado con los mejores hiperparámetros.

In [13]:
gs.best_estimator_

,criterion,'gini'
,splitter,'best'
,max_depth,6
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


**Ejemplo 2:** la clase `RandomizedSearchCV` se puede usar casi de la misma manera, solo que esta vez se debe escoger un número de combinaciones a evaluar; las cuales se escogeran de manera aleatoria.

In [15]:
clf = DecisionTreeClassifier()

# define los valores que usarás en la búsqueda del
# hiperparametro C
param_dist = {
    "max_depth": (3, 6, 12, 18),
    "min_samples_leaf": (1, 2, 3),
    "criterion": ["gini", "entropy"]
}

# Utiliza RandomizedSearchCV
rs = RandomizedSearchCV(clf, param_dist, scoring="f1", cv=5, verbose=1, n_jobs=-1, n_iter=10, random_state=426) #random_statte = últimos 3 digitos de mi cédula para reproducibilidad

t0 = time.time()
rs.fit(X_train, y_train)
print("Tiempo de búsqueda: {:.3f}s".format(time.time() - t0))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Tiempo de búsqueda: 7.740s


In [16]:
rs.best_score_

np.float64(0.9513984697470935)

Podemos ver que aunque se demoró mucho menos, el resultado no es tan bueno.

# Experimento 2

Hyperopt es una biblioteca de Python para optimización de hiperparámetros que resulta  más eficiente que métodos tradicionales como GridSearch o RandomizedSearch. A diferencia de estos, que prueban combinaciones de forma exhaustiva o aleatoria sin aprender del proceso, Hyperopt utiliza algoritmos como TPE (Tree-structured Parzen Estimator) para tomar decisiones basadas en resultados anteriores, lo que permite explorar el espacio de parámetros de manera más inteligente y con menos pruebas, ahorrando tiempo y recursos computacionales.

In [89]:
#%pip install --upgrade numpy hyperopt setuptools

from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


In [94]:

# Definir el espacio de búsqueda de hiperparámetros
space = {
    'max_depth': hp.choice('max_depth', [3, 6, 12, 18]),
    'min_samples_leaf': hp.choice('min_samples_leaf', [1, 2, 3]),
    'criterion': hp.choice('criterion', ['gini', 'entropy'])
}


In [107]:
# Definir la función objetivo para Hyperopt
def objective(params):
    clf = DecisionTreeClassifier(
        max_depth=params['max_depth'],
        min_samples_leaf=params['min_samples_leaf'],
        criterion=params['criterion']
    )
    score = cross_val_score(clf, X_train, y_train, scoring='f1', cv=5).mean()
    return {'loss': -score, 'status': STATUS_OK}

# Inicia el cronómetro
t0 = time.time()

# Búsqueda de hiperparámetros con Hyperopt
trials = Trials()
best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=20,
    trials=trials,
    rstate=np.random.default_rng(426), 
    show_progressbar=False  # Oculta la barra de progreso
    )

print("Tiempo de búsqueda: {:.3f}s".format(time.time() - t0))

# Mapear los índices a los valores reales
best_params = {
    'max_depth': [3, 6, 12, 18][best['max_depth']],
    'min_samples_leaf': [1, 2, 3][best['min_samples_leaf']],
    'criterion': ['gini', 'entropy'][best['criterion']]
}

clf_best = DecisionTreeClassifier(**best_params)
clf_best.fit(X_train, y_train)
y_pred = clf_best.predict(X_test)

# Medimos el score f1
test_f1 = f1_score(y_test, y_pred)
print("F1-score para los parámetros hallados por Hyperopt:", test_f1)

Tiempo de búsqueda: 0.790s
F1-score para los parámetros hallados por Hyperopt: 0.9444444444444444


Nos damos cuenta que el resultados del score f1 obtenidos por GridSearch, RandomisedSearch y el fmin de Hyperopt son muy parecidos, cercanos a 0.95 . Como esta calificación de por sí es excelente, no esperábamos una mejoría por parte de los parámetros obtenidos por el fmin de hyperopt, pues ya estaba demostrado que el espacio de búsqueda definido es adecuado y que el problema no es especialmente difícil para los modelos más simples en estos datos.